segmentation 에 있는 patient 들의 segmentation image 를 만든다.

In [1]:
import os
import glob
import pydicom
import nibabel as nib
import pandas as pd
import numpy as np
from pydicom.pixel_data_handlers.util import apply_voi_lut
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns

from tqdm import tqdm

%load_ext autoreload
%autoreload 2

import sys
sys.path.append("..")

from utils.dcm_utils import *
from utils.nii_utils import *

from PIL import Image

import csv

In [2]:
DATA_DIR = "/Volumes/SSD970/"
TRAIN_SAGITTAL_DIR = os.path.join(DATA_DIR, "sagittal_images")
TRAIN_IMAGE_DIR = os.path.join(DATA_DIR, "train_images")
TRAIN_SEG_DIR = os.path.join(DATA_DIR, "segmentations")

In [3]:
patient_nii_files = glob.glob(TRAIN_SEG_DIR + "/**")
len(patient_nii_files)

87

In [4]:
patient_nii_files

['/Volumes/SSD970/segmentations/1.2.826.0.1.3680043.10633.nii',
 '/Volumes/SSD970/segmentations/1.2.826.0.1.3680043.10921.nii',
 '/Volumes/SSD970/segmentations/1.2.826.0.1.3680043.11827.nii',
 '/Volumes/SSD970/segmentations/1.2.826.0.1.3680043.11988.nii',
 '/Volumes/SSD970/segmentations/1.2.826.0.1.3680043.12281.nii',
 '/Volumes/SSD970/segmentations/1.2.826.0.1.3680043.12292.nii',
 '/Volumes/SSD970/segmentations/1.2.826.0.1.3680043.12833.nii',
 '/Volumes/SSD970/segmentations/1.2.826.0.1.3680043.1363.nii',
 '/Volumes/SSD970/segmentations/1.2.826.0.1.3680043.14267.nii',
 '/Volumes/SSD970/segmentations/1.2.826.0.1.3680043.1480.nii',
 '/Volumes/SSD970/segmentations/1.2.826.0.1.3680043.15206.nii',
 '/Volumes/SSD970/segmentations/1.2.826.0.1.3680043.1542.nii',
 '/Volumes/SSD970/segmentations/1.2.826.0.1.3680043.1573.nii',
 '/Volumes/SSD970/segmentations/1.2.826.0.1.3680043.16092.nii',
 '/Volumes/SSD970/segmentations/1.2.826.0.1.3680043.16919.nii',
 '/Volumes/SSD970/segmentations/1.2.826.0.1.

In [5]:
header = [
    'UID', 'sagittal_index',
    'z_spacing','pixel_spacing','aspect',
    'width','height', 'is_flip'
]

In [6]:
f = open('meta_segmentation_sagittal.csv','w',newline='')
writer = csv.writer(f)
writer.writerow(header)

72

In [7]:
def save_sagittal_image(UID):

    patient_sagittal_dir = os.path.join(TRAIN_SAGITTAL_DIR, UID)

    if os.path.exists(patient_sagittal_dir) == False:
        os.mkdir(patient_sagittal_dir)

    imgs, image_positions, image_orientations, pixel_spacings, slice_thicknesses, is_flip = read_patient_dcm(os.path.join(TRAIN_IMAGE_DIR, UID))

    z_spacing = image_positions[0, 2] - image_positions[1, 2]
    pixel_spacing = pixel_spacings[0, 0]
    aspect = z_spacing / pixel_spacing # ㅇㅕ기를 원래 round 했다. repeat 하길래서 round 시킨걸 계속 갖고 있었네 ㅜㅜ
    imgs = normalize_hu(imgs)

    img_width = imgs.shape[1]
    # 그냥 int 를 취했다 round 하지 않았다.
    img_height = int(imgs.shape[0] * aspect)

    for sagittal_index in range(imgs.shape[2]):
        sagittal = imgs[:, :, sagittal_index]
        sagittal = Image.fromarray(np.uint8(sagittal * 255) , 'L')

        sagittal = sagittal.resize([img_width, img_height])
        sagittal.save(os.path.join(patient_sagittal_dir, f"{sagittal_index}.jpeg"))

        writer.writerow([
            UID, sagittal_index,
            z_spacing, pixel_spacing, aspect,
            img_width, img_height, int(is_flip)
        ])

    return True

In [8]:
# patient_train_dir = patient_dirs[0]
patient_nii_files_iter = tqdm(patient_nii_files)
for nii_path in patient_nii_files_iter:
    UID = get_uid_from_niipath(nii_path)
    save_sagittal_image(UID)

    patient_nii_files_iter.set_description(UID)

  0%|          | 0/87 [00:00<?, ?it/s]

total slices 429


1.2.826.0.1.3680043.10633:   1%|          | 1/87 [00:12<18:35, 12.97s/it]

total slices 339


1.2.826.0.1.3680043.10921:   2%|▏         | 2/87 [00:19<13:14,  9.35s/it]

...

1.2.826.0.1.3680043.8744:  98%|█████████▊| 85/87 [10:59<00:11,  5.66s/it]

total slices 303


1.2.826.0.1.3680043.8884:  99%|█████████▉| 86/87 [11:05<00:05,  5.83s/it]

total slices 170


1.2.826.0.1.3680043.9926: 100%|██████████| 87/87 [11:10<00:00,  7.70s/it]


In [9]:
f.close

<function TextIOWrapper.close()>